In [22]:
import os
import ipynbname

chemin_notebook = ipynbname.path()
dossier_notebook = os.path.dirname(chemin_notebook)
os.chdir(dossier_notebook)
os.chdir('../..')
print("Répertoire actuel :", os.getcwd())

Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git


In [23]:
from Utils.embedding.old.old_embedding import *

On va modifier l'embedding des types. On va réaliser un autoencodeur pour représenter la répart des types des pokes dans un espace 3 (et non 18). On garde un ordre dans ces représentations, car savoir quel type correspond à quel poké est important. Aussi on peut rajouter le nb_fainted, mais un vecteur 12 dim avec les positions des pokémons fainted, ça peut aider je pense à éviter les fausses actions.

In [24]:
#Instancier l'encodeur du type 
from Utils.embedding.old.type_autoencodeur import TypeAutoencoder

encodeur_type = TypeAutoencoder(encoded_size=4)

# Charger les poids sauvegardés
encodeur_type.load_state_dict(torch.load("Utils/embedding/old/type_autoencoder.pth", map_location='mps'))
encodeur_type.eval()

TypeAutoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=17, out_features=12, bias=True)
    (1): ReLU()
    (2): Linear(in_features=12, out_features=4, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=4, out_features=12, bias=True)
    (1): ReLU()
    (2): Linear(in_features=12, out_features=17, bias=True)
    (3): Sigmoid()
  )
)

Encodeur créé. Maintenant on remplace les types par les vecteurs encodés

In [25]:
############## DEFINIR L'EMBEDDING ##################
import numpy as np
from gymnasium.spaces import Space, Box
from poke_env.player import Gen4EnvSinglePlayer
from poke_env.data import GenData
import numpy as np
from poke_env.environment.abstract_battle import AbstractBattle
import torch
from gymnasium.spaces import Discrete, Box

# Initialiser GenData pour la génération souhaitée (par exemple, génération 4)
gen_data = GenData.from_gen(4)

# Accéder au tableau des types
type_chart = gen_data.type_chart


class embedding_Player(Gen4EnvSinglePlayer):
    def __init__(self,model_path=None, **kwargs):
        super().__init__(**kwargs)
        if model_path is not None :
            self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
            print(f"📥 Modèle chargé depuis {model_path}")

        self.action_space = Discrete(9)  # ✅ attribut classique
    
    #Toujours mêmes valeurs de reward
    def calc_reward(self, last_battle, current_battle) -> float:
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )

    def embed_battle(self, battle )  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        pokemon_types_compressed = []
        #Pokemon types  
        pokemon_types = obtain_pokemon_types(battle)
        pokemon_types = torch.tensor(obtain_pokemon_types(battle)).view(12, 17)  # 6 pokés par team = 12 vecteurs
        with torch.no_grad():
            for i in range(12) :
                vec = pokemon_types[i].unsqueeze(0)
                encoded = encodeur_type.encoder(vec.float()) 
                pokemon_types_compressed.append(encoded.squeeze(0))

        # Final vector with 10 components
        compressed_flat = torch.cat(pokemon_types_compressed).numpy()
        final_vector = np.concatenate(
            [
                moves_real_power,
                compressed_flat,
            ]
        )

        return np.float32(final_vector)

    def describe_embedding(self) -> Space:
        low = (
            [-1] * 4 +          # real power
            [0] * 24 +         # my team types
            [0] * 24           # opponent team types
        )
        high = (
            [3] * 4 +           # real power
            [1] * 24 +         # my team types
            [1] * 24           # opponent team types
        )

        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32
        )
    
    def action_to_move(self, action: int, battle: AbstractBattle):
        order = super().action_to_move(action, battle)
        order.dynamax = False  # 🔥 désactive Dynamax pour toutes les actions
        return order

In [26]:
########### DEFINIR MODELE ##################
import torch
import torch.nn as nn
import torch.nn.functional as F

#Réseau perceptron à une couche cachée, sortie linéaire, f activation = Relu,
class DQNModel(nn.Module):
    def __init__(self, input_dim, n_actions):
        super(DQNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=64, kernel_size=3, padding=1)
        self.conv2= nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, padding=1)
        self.out = nn.Linear(32, n_actions)

    def forward(self, x):
        real_power = x[:4]
        types = x[4:] 
        real_power_padded = F.pad(real_power, (0, 14))  # [17]
        real_power_padded = real_power_padded.unsqueeze(0)  # [1, 17] 
        sequence = torch.cat([real_power_padded, types], dim=0)
        sequence = sequence.T.unsqueeze(0)     
        types = types.view(12, 17) 
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return self.out(x)

***TRAINING***

In [27]:

opponent = RandomPlayer(battle_format="gen4randombattle")
from stable_baselines3.common.vec_env import DummyVecEnv

# instanciation du player
train_env_raw = embedding_Player(
    battle_format="gen4randombattle",
    opponent=RandomPlayer(battle_format="gen4randombattle"),
    start_challenging=True
)

# wrap dans DummyVecEnv (SB3 attend un vecteur d’envs, même pour un seul)
train_env = DummyVecEnv([lambda: train_env_raw])

In [28]:
env = train_env

model = DQN(
    policy="MlpPolicy",
    env=env,
    learning_rate=2.5e-4,
    buffer_size=10000,
    learning_starts=1000,
    batch_size=32,
    gamma=0.5,
    train_freq=1,
    target_update_interval=1,
    exploration_fraction=1.0,
    exploration_final_eps=0.05,
    policy_kwargs=dict(activation_fn=nn.ReLU, net_arch=[128, 64])
)


In [29]:
steps = 50000
callback = CustomTQDMCallback(total_timesteps = steps, check_freq=1000, verbose=1)
model.learn(
    total_timesteps=steps,
    callback=callback
)
model.save("Players/embedding_2")

📊 Training progress:   0%|          | 0/50000 [00:00<?, ?it/s]

2025-04-14 01:49:59,260 - RandomPlayer 7 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p1a: Vileplume', 'Solar Beam', 'p2a: Dragonite', '[from] lockedmove'] in battle battle-gen4randombattle-8134 turn 6
2025-04-14 01:49:59,386 - RandomPlayer 7 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p2a: Snorlax', 'Pursuit', 'p1a: Ninjask', '[from] Pursuit'] in battle battle-gen4randombattle-8134 turn 45
2025-04-14 01:50:00,227 - RandomPlayer 7 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p1a: Skuntank', 'Pursuit', 'p2a: Castform', '[from] Pursuit'] in battle battle-gen4randombattle-8139 turn 15
2025-04-14 01:50:00,515 - RandomPlayer 7 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p1a: Rayquaza', 'Outrage', 'p2a: Unown', '[from] lockedmove'] in battle battle-gen4randombattle-8141 turn 23
2025-04-14 01:50:00,641 - RandomPlayer 7 - WARNING 

In [30]:
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env

class EmbeddingTestPlayer(Gen4EnvSinglePlayer):

    def __init__(self, model_path="Players/embedding_2", **kwargs):
        super().__init__(**kwargs)
        self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
        print(f"📥 Modèle chargé depuis {model_path}")
        print("Répertoire actuel :", os.getcwd())

        #Toujours mêmes valeurs de reward
    def calc_reward(self, last_battle, current_battle) -> float:
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )

    def embed_battle(self, battle )  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        pokemon_types_compressed = []
        #Pokemon types  
        pokemon_types = obtain_pokemon_types(battle)
        pokemon_types = torch.tensor(obtain_pokemon_types(battle)).view(12, 17)  # 6 pokés par team = 12 vecteurs
        with torch.no_grad():
            for i in range(12) :
                vec = pokemon_types[i].unsqueeze(0)
                encoded = encodeur_type.encoder(vec.float()) 
                pokemon_types_compressed.append(encoded.squeeze(0))

        # Final vector with 10 components
        compressed_flat = torch.cat(pokemon_types_compressed).numpy()
        final_vector = np.concatenate(
            [
                moves_real_power,
                compressed_flat,
            ]
        )

        return np.float32(final_vector)

    def describe_embedding(self) -> Space:
        low = (
            [-1] * 4 +          # real power
            [0] * 24 +         # my team types
            [0] * 24           # opponent team types
        )
        high = (
            [3] * 4 +           # real power
            [1] * 24 +         # my team types
            [1] * 24           # opponent team types
        )

        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32
        )
    
    def action_to_move(self, action, battle):
        moves = battle.available_moves
        switches = battle.available_switches
        total_actions = len(moves) + len(switches)

        #print(f"🔢 DQN → action={action} | #moves={len(moves)} | #switches={len(switches)} | total={total_actions}")

        if 0 <= action < len(moves):
            return self.create_order(moves[action])
        elif len(moves) <= action < total_actions:
            return self.create_order(switches[action - len(moves)])
        else:
            #print("❌ Action hors bornes ! Fallback sur move aléatoire")
            return self.choose_random_move(battle)
        
    def predict(self, obs):
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        q_values = self.model.q_net(obs_tensor)
        print(f"📊 Q-values : {q_values.detach().numpy().flatten()}")
        action = int(torch.argmax(q_values).item())
        return action

In [31]:
print("Répertoire actuel :", os.getcwd())
opponent = RandomPlayer(battle_format="gen4randombattle")
eval_env = EmbeddingTestPlayer(
    battle_format="gen4randombattle", opponent=opponent, start_challenging=True
)

Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git
📥 Modèle chargé depuis Players/embedding_2
Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git


In [32]:
n_eval_episodes = 40
rewards = []
wins = 0

obs, _ = eval_env.reset()
for _ in tqdm(range(n_eval_episodes)):
    done = False
    total_reward = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, _ = eval_env.step(action)
        done = terminated or truncated

        total_reward += reward
        if done and reward > 0:
            wins += 1

    rewards.append(total_reward)
    obs, _ = eval_env.reset()

print(f"✅ {wins} victoires sur {n_eval_episodes} matchs")
print(f"🎯 Reward moyen : {sum(rewards) / len(rewards):.2f}")

  0%|          | 0/40 [00:00<?, ?it/s]

2025-04-14 01:53:13,530 - RandomPlayer 8 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p2a: Salamence', 'Outrage', 'p1a: Dusknoir', '[from] lockedmove'] in battle battle-gen4randombattle-9092 turn 8
2025-04-14 01:53:13,533 - RandomPlayer 8 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p2a: Salamence', 'Outrage', 'p1a: Dusknoir', '[from] lockedmove'] in battle battle-gen4randombattle-9092 turn 9
2025-04-14 01:53:13,727 - RandomPlayer 8 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p2a: Gyarados', 'Dragon Dance', 'p2a: Gyarados', '[from] move: Sleep Talk'] in battle battle-gen4randombattle-9093 turn 35
2025-04-14 01:53:14,021 - RandomPlayer 8 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p1a: Parasect', 'Pursuit', 'p2a: Gyarados', '[from] Pursuit'] in battle battle-gen4randombattle-9094 turn 91
2025-04-14 01:53:14,024 - RandomPlay

✅ 35 victoires sur 40 matchs
🎯 Reward moyen : 28.83


Adversarial Training

In [33]:
class AdversarialTrainPlayer(RandomPlayer):

    def __init__(self, model_path="Players/embedding_2", **kwargs):
        super().__init__(**kwargs)
        self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
        print(f"📥 Modèle chargé depuis {model_path}")

    def embed_battle(self, battle )  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        pokemon_types_compressed = []
        #Pokemon types  
        pokemon_types = obtain_pokemon_types(battle)
        pokemon_types = torch.tensor(obtain_pokemon_types(battle)).view(12, 17)  # 6 pokés par team = 12 vecteurs
        with torch.no_grad():
            for i in range(12) :
                vec = pokemon_types[i].unsqueeze(0)
                encoded = encodeur_type.encoder(vec.float()) 
                pokemon_types_compressed.append(encoded.squeeze(0))

        # Final vector with 10 components
        compressed_flat = torch.cat(pokemon_types_compressed).numpy()
        final_vector = np.concatenate(
            [
                moves_real_power,
                compressed_flat,
            ]
        )

        return np.float32(final_vector)
    
    #action_to_move suppr
        
    def predict(self, obs):
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        q_values = self.model.q_net(obs_tensor)
        print(f"📊 Q-values : {q_values.detach().numpy().flatten()}")
        action = int(torch.argmax(q_values).item())
        return action
    
    def choose_move(self, battle):
        obs = self.embed_battle(battle)
        device = "mps" if torch.backends.mps.is_available() else "cpu"
        obs_tensor = torch.tensor(obs, dtype=torch.float32, device=device).unsqueeze(0)
        with torch.no_grad():
            q_values = self.model.q_net(obs_tensor)
            action = int(torch.argmax(q_values).item())

        moves = battle.available_moves
        switches = battle.available_switches
        total_actions = len(moves) + len(switches)

        if 0 <= action < len(moves):
            return self.create_order(moves[action])
        elif len(moves) <= action < total_actions:
            return self.create_order(switches[action - len(moves)])
        else:
            return self.choose_random_move(battle)

In [34]:

train_env_raw = embedding_Player(
    battle_format="gen4randombattle",
    opponent=AdversarialTrainPlayer(battle_format="gen4randombattle"),
    start_challenging=True
)
train_env = DummyVecEnv([lambda: train_env_raw])

adv_learn_model = DQN.load("Players/embedding_2", env=train_env, device="mps" if torch.backends.mps.is_available() else "cpu")

# Puis on continue l'entraînement
adv_learn_model.learn(total_timesteps=100)

adv_learn_model.save("Players/embedding_bidon")

📥 Modèle chargé depuis Players/embedding_2


2025-04-14 01:53:20,077 - AdversarialTrain 2 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p2a: Dragonite', 'Outrage', 'p1a: Cresselia', '[from] lockedmove'] in battle battle-gen4randombattle-9128 turn 46


In [36]:
opponent = AdversarialTrainPlayer(battle_format="gen4randombattle")

eval_env = EmbeddingTestPlayer(
    battle_format="gen4randombattle", opponent=opponent, start_challenging=True, model_path="Players/embedding_bidon"
)

📥 Modèle chargé depuis Players/embedding_2
📥 Modèle chargé depuis Players/embedding_bidon
Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git


In [37]:
n_eval_episodes = 40
rewards = []
wins = 0

obs, _ = eval_env.reset()
for _ in tqdm(range(n_eval_episodes)):
    done = False
    total_reward = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, _ = eval_env.step(action)
        done = terminated or truncated

        total_reward += reward
        if done and reward > 0:
            wins += 1

    rewards.append(total_reward)
    obs, _ = eval_env.reset()

print(f"✅ {wins} victoires sur {n_eval_episodes} matchs")
print(f"🎯 Reward moyen : {sum(rewards) / len(rewards):.2f}")

  0%|          | 0/40 [00:00<?, ?it/s]

2025-04-14 01:55:09,310 - AdversarialTrain 4 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p1a: Garchomp', 'Outrage', 'p2a: Nidoqueen', '[from] lockedmove'] in battle battle-gen4randombattle-9135 turn 2
2025-04-14 01:55:09,323 - AdversarialTrain 4 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p1a: Garchomp', 'Outrage', 'p2a: Nidoqueen', '[from] lockedmove'] in battle battle-gen4randombattle-9135 turn 4
2025-04-14 01:55:14,090 - AdversarialTrain 4 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p1a: Salamence', 'Outrage', 'p2a: Fearow', '[from] lockedmove'] in battle battle-gen4randombattle-9148 turn 34
2025-04-14 01:55:14,098 - AdversarialTrain 4 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p1a: Salamence', 'Outrage', 'p2a: Zapdos', '[from] lockedmove'] in battle battle-gen4randombattle-9148 turn 35
2025-04-14 01:55:16,527 - Adve

✅ 22 victoires sur 40 matchs
🎯 Reward moyen : 3.52


Je vais l'affronter moi 

In [ ]:
from Utils.DQ_simple import DQ_simple 
class DQ_simpleemb3(DQ_simple) :
    def __init__(self, model_path = "Players/embedding_3", battle_format="gen4randombattle"):
        super().__init__(battle_format=battle_format)
        
        self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
        print("📥 Modèle DQN chargé :", self.model)
    
    def embed_battle(self, battle )  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
            moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        #Pokemon types  
        pokemon_types_compressed = []
        pokemon_types = obtain_pokemon_types(battle)
        pokemon_types = torch.tensor(obtain_pokemon_types(battle)).view(12, 17)  # 6 pokés par team = 12 vecteurs
        with torch.no_grad():
            for i in range(12) :
                vec = pokemon_types[i].unsqueeze(0)
                encoded = encodeur_type.encoder(vec.float()) 
                pokemon_types_compressed.append(encoded.squeeze(0))

        # Final vector with 10 components
        compressed_flat = torch.cat(pokemon_types_compressed).numpy()
        final_vector = np.concatenate(
            [
                moves_real_power,
                compressed_flat,
            ]
        )

        return np.float32(final_vector)

    def choose_move(self, battle):
        print("👉 choose_move appelée !")
        # 🔍 Debug : Voir les moves disponibles
        print(f"🔍 Moves disponibles : {[move.id for move in battle.available_moves]}")

        # Obtenir l'observation de l'état du combat
        obs = self.embed_battle(battle)
        print("📊 Observation de l'état :", obs)

        # Transformer en format PyTorch
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        print("📊 Tensor pour le modèle :", obs_tensor)

        # Prédire l'action avec le modèle DQN
        action = int(self.model.predict(obs_tensor, deterministic=True)[0])
        
        print("🎯 Action choisie par DQN :", action)
        moves = battle.available_moves
        switches = battle.available_switches
        total_actions = len(moves) + len(switches)
        self.create_order.dynamax = False

        if 0 <= action < len(moves):
            return self.create_order(moves[action])
        elif len(moves) <= action < total_actions:
            return self.create_order(switches[action - len(moves)])
        else:
            print("XXXX   : Random Moove ")
            return self.choose_random_move(battle)

In [ ]:

bott_player = DQ_simpleemb3()

#await bott_player.send_challenges("chewba", n_challenges=1)

📥 Modèle DQN chargé : <stable_baselines3.dqn.dqn.DQN object at 0x3c16b4150>
📥 Modèle DQN chargé : <stable_baselines3.dqn.dqn.DQN object at 0x3a2e0cad0>
👉 choose_move appelée !
🔍 Moves disponibles : ['earthquake', 'woodhammer', 'rockpolish', 'stoneedge']
📊 Observation de l'état : [ 2.          1.2         0.          0.5        -0.861185   -1.180399
  1.6741319   1.1654061  -0.9784033  -0.9923588   1.0365235   0.748731
 -0.8383499  -0.9495262   0.9597749   0.7300904  -0.9644627  -1.1416496
  1.574438    0.9674219  -0.766906   -1.004926    1.0535971   0.8476578
 -1.3287634  -1.2843244   1.6307695   1.2348181  -1.4173565  -1.2791059
  1.5215273   1.1027566  -0.70552427 -0.8069108   0.70577747  0.5724884
 -0.70552427 -0.8069108   0.70577747  0.5724884  -0.70552427 -0.8069108
  0.70577747  0.5724884  -0.70552427 -0.8069108   0.70577747  0.5724884
 -0.70552427 -0.8069108   0.70577747  0.5724884 ]
📊 Tensor pour le modèle : tensor([[ 2.0000,  1.2000,  0.0000,  0.5000, -0.8612, -1.1804,  1.6741

/var/folders/h3/sk18hvh55gqbml8209nmkrlc0000gq/T/ipykernel_95459/548507695.py:62: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  action = int(self.model.predict(obs_tensor, deterministic=True)[0])


👉 choose_move appelée !
🔍 Moves disponibles : ['earthquake', 'woodhammer', 'rockpolish', 'stoneedge']
📊 Observation de l'état : [ 2.          1.2         0.          0.5        -0.861185   -1.180399
  1.6741319   1.1654061  -0.9784033  -0.9923588   1.0365235   0.748731
 -0.8383499  -0.9495262   0.9597749   0.7300904  -0.9644627  -1.1416496
  1.574438    0.9674219  -0.766906   -1.004926    1.0535971   0.8476578
 -1.3287634  -1.2843244   1.6307695   1.2348181  -1.4173565  -1.2791059
  1.5215273   1.1027566  -0.70552427 -0.8069108   0.70577747  0.5724884
 -0.70552427 -0.8069108   0.70577747  0.5724884  -0.70552427 -0.8069108
  0.70577747  0.5724884  -0.70552427 -0.8069108   0.70577747  0.5724884
 -0.70552427 -0.8069108   0.70577747  0.5724884 ]
📊 Tensor pour le modèle : tensor([[ 2.0000,  1.2000,  0.0000,  0.5000, -0.8612, -1.1804,  1.6741,  1.1654,
         -0.9784, -0.9924,  1.0365,  0.7487, -0.8383, -0.9495,  0.9598,  0.7301,
         -0.9645, -1.1416,  1.5744,  0.9674, -0.7669, -1.004